In [1]:
import os
import shutil

import pandas as pd
import pydotplus
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.preprocessing import OneHotEncoder

In [2]:
IMAGES_DIR = 'images'
shutil.rmtree(IMAGES_DIR, ignore_errors=True)
os.mkdir(IMAGES_DIR)

In [3]:
MARKS_DIR = 'marks'
MARKS_INFO = os.path.join(MARKS_DIR, 'marks-info.txt')
MARKS_DATA = os.path.join(MARKS_DIR, 'marks.csv')

In [4]:
with open(MARKS_INFO, 'r', encoding='cp1251') as f:
    info = f.read().replace('\n', '')

> {{info}}

In [5]:
marks_data = pd.read_csv(MARKS_DATA)

In [6]:
marks_data.T

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
PUPIL_SEX,F,F,F,M,M,M,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
PUPIL_CLASS,8A,8A,8A,8A,8A,8A,8A,8A,8A,8A,...,8A,8A,8A,8A,8A,8A,8A,8A,8A,8A
TEACHER_RIGHT,65,70,85,55,40,65,67,60,100,100,...,90,95,50,60,70,80,85,90,95,55
TEACHER_CHK,0,4,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
TEACHER_QUEST,4,0,0,0,2,3,0,2,0,0,...,0,1,2,0,0,0,0,0,0,0
TEACHER_CORR,2,4,4,1,0,4,1,0,2,2,...,0,2,6,4,2,2,1,0,3,0
PUPIL_CORR,1,0,3,8,3,4,3,5,0,3,...,1,1,0,5,1,1,2,1,1,3
PUPIL_STRIP,6,4,5,3,4,8,3,9,2,1,...,0,0,5,0,4,7,2,1,2,3
FINALMARK,4-,3,4,3,2,4-,4-,3,5-,4,...,4,5-,3,3,4-,4-,4,4,5-,3


In [7]:
marks_train, marks_test = marks_data.drop(['FINALMARK'], axis=1), marks_data['FINALMARK']

In [8]:
CAT_FEATURES = ['PUPIL_SEX', 'PUPIL_CLASS']

In [9]:
num_features = marks_train.drop(CAT_FEATURES, axis=1)

In [10]:
FEATURES = list(num_features.columns.values)
FEATURES

['TEACHER_RIGHT',
 'TEACHER_CHK',
 'TEACHER_QUEST',
 'TEACHER_CORR',
 'PUPIL_CORR',
 'PUPIL_STRIP']

In [11]:
marks_encoder = OneHotEncoder(handle_unknown='ignore')

In [12]:
cat_features = marks_encoder.fit_transform(marks_train[CAT_FEATURES])

In [13]:
num_features.T

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
TEACHER_RIGHT,65,70,85,55,40,65,67,60,100,100,...,90,95,50,60,70,80,85,90,95,55
TEACHER_CHK,0,4,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
TEACHER_QUEST,4,0,0,0,2,3,0,2,0,0,...,0,1,2,0,0,0,0,0,0,0
TEACHER_CORR,2,4,4,1,0,4,1,0,2,2,...,0,2,6,4,2,2,1,0,3,0
PUPIL_CORR,1,0,3,8,3,4,3,5,0,3,...,1,1,0,5,1,1,2,1,1,3
PUPIL_STRIP,6,4,5,3,4,8,3,9,2,1,...,0,0,5,0,4,7,2,1,2,3


In [14]:
cat_features = cat_features.toarray()

In [15]:
one_hot_features = {name: features for name, features in zip(marks_encoder.get_feature_names(), cat_features.T)}

In [16]:
one_hot_data = pd.DataFrame(one_hot_features)
one_hot_data.head()

,x0_F,x0_M,x1_8A,x1_8B
0,1.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0
2,1.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0
4,0.0,1.0,1.0,0.0


In [17]:
mark_features = pd.concat([num_features, one_hot_data], axis=1, sort=False)
mark_features.head()

,TEACHER_RIGHT,TEACHER_CHK,TEACHER_QUEST,TEACHER_CORR,PUPIL_CORR,PUPIL_STRIP,x0_F,x0_M,x1_8A,x1_8B
0,65,0,4,2,1,6,1.0,0.0,1.0,0.0
1,70,4,0,4,0,4,1.0,0.0,1.0,0.0
2,85,0,0,4,3,5,1.0,0.0,1.0,0.0
3,55,0,0,1,8,3,0.0,1.0,1.0,0.0
4,40,1,2,0,3,4,0.0,1.0,1.0,0.0


In [18]:
feature_names = list(mark_features.columns.values)

**Разные варианты глубины**

In [19]:
clf_marks_tree = DecisionTreeClassifier(
    criterion='entropy', 
#     max_depth=3, 
    random_state=17,
)

In [20]:
clf_marks_tree.fit(mark_features, marks_test)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [21]:
def tree_graph_to_png(tree, feature_names, png_file_to_save, class_names=None):
    tree_str = export_graphviz(tree, feature_names=feature_names, 
                                       class_names=class_names,
                                     filled=True, out_file=None)
    graph = pydotplus.graph_from_dot_data(tree_str)  
    graph.write_png(png_file_to_save)

In [22]:
marks_image = os.path.join(IMAGES_DIR, 'marks.png')

In [23]:
tree_graph_to_png(tree=clf_marks_tree, 
                  feature_names=feature_names, 
                  class_names=clf_marks_tree.classes_, 
                  png_file_to_save=marks_image)

In [24]:
marks_image

'images/marks.png'

![](images/marks.png)